In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir('../../notebook_format')
from formats import load_style
load_style()

In [2]:
# As usual, a bit of setup
os.chdir(path)

import time
import numpy as np
import cs231n.layers as layers
import matplotlib.pyplot as plt
from cs231n.solver import Solver
from cs231n.data_utils import get_CIFAR10_data
from cs231n.classifiers.fc_net import FullyConnectedNet
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """returns relative error"""
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Batch Normalization
One way to make deep networks easier to train is to use more sophisticated optimization procedures such as SGD+momentum, RMSProp, or Adam. Another strategy is to change the architecture of the network to make it easier to train. One idea along these lines is batch normalization which was recently proposed by [3].

The idea is relatively straightforward. Machine learning methods tend to work better when their input data consists of uncorrelated features with zero mean and unit variance. When training a neural network, we can preprocess the data before feeding it to the network to explicitly decorrelate its features; this will ensure that the first layer of the network sees data that follows a nice distribution. However even if we preprocess the input data, the activations at deeper layers of the network will likely no longer be decorrelated and will no longer have zero mean or unit variance since they are output from earlier layers in the network. Even worse, during the training process the distribution of features at each layer of the network will shift as the weights of each layer are updated.

The authors of [3] hypothesize that the shifting distribution of features inside deep neural networks may make training deep networks more difficult. To overcome this problem, [3] proposes to insert batch normalization layers into the network. At training time, a batch normalization layer uses a minibatch of data to estimate the mean and standard deviation of each feature. These estimated means and standard deviations are then used to center and normalize the features of the minibatch. A running average of these means and standard deviations is kept during training, and at test time these running averages are used to center and normalize features.

In the implementation, applying this technique usually amounts to insert the BatchNorm layer immediately after fully connected layers (or convolutional layers, as we’ll soon see), and before non-linearities.

It is possible that this normalization strategy could reduce the representational power of the network, since it may sometimes be optimal for certain layers to have features that are not zero-mean or unit variance. To this end, the batch normalization layer includes learnable shift and scale parameters for each feature dimension.

[3] [Sergey Ioffe and Christian Szegedy, "Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift", ICML 2015](http://arxiv.org/abs/1502.03167).

In [3]:
# Load the (preprocessed) CIFAR10 data
data = get_CIFAR10_data()
for k, v in data.items():
    print( '%s: %s' % ( k, v.shape ) )

X_test: (1000, 3, 32, 32)
y_train: (49000,)
y_test: (1000,)
X_train: (49000, 3, 32, 32)
y_val: (1000,)
X_val: (1000, 3, 32, 32)


## Batch normalization: Forward

A nice writeup on this part [Blog: Batch Normalization Layer](https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html).

In the file `cs231n/layers.py`, implement the batch normalization forward pass in the function `batchnorm_forward`. Once you have done so, run the following to test your implementation.

In [5]:
N, D1, D2, D3 = 200, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum( 0, X.dot(W1) ).dot(W2)

In [7]:
eps = 1e-5
gamma = 1
beta = 1

N, D = a.shape

# step1: calculate mean
mu = np.mean( a, axis = 0 )

# step2: subtract the mean vector of every input data
# from the data
a_mu = a - mu

# step3: following the lower branch - calculation denominator
squared = a_mu ** 2

# step4: calculate variance
variance = np.mean( squared, axis = 0 )

# step5: add eps for numerical stability, then sqrt
sqrtvar = np.sqrt( variance + eps )

# step6: invert sqrtvar
sqrtvar_invert = 1 / sqrtvar

# step7: execute normalization
xhat = a_mu * sqrtvar_invert

# step8: the two transformation steps
gammax = gamma * xhat

# step9
out = gammax + beta

# cache intermediate values

In [47]:
dout = a.copy()
# get the dimensions of the input/output
N, D = a.shape

# step 9
# because the summation of beta during the forward pass 
# is a row-wise summation, during the backward pass we 
# need to sum up the gradient, this easier to understand if you've
# print out the shape
dbeta = np.sum( dout, axis = 0 )
dgammax = dout # not necessary, but more understandable

# step 8
dgamma = np.sum( dgammax * xhat, axis = 0 )
dx_hat  = dgammax * gamma

# step 7
dx_mu1 = dxhat * sqrtvar_invert
dsqrtvar_invert = np.sum( dx_hat * a_mu, axis = 0 )

# step 6
dsqrtvar = - sqrtvar ** -2 * dsqrtvar_invert

# step 5
dvariance = 0.5 * ( variance + eps ) ** -0.5 * dsqrtvar

# step 4
dsquared = 1 / N * np.ones(( N, D )) * dvariance

# step 3
dx_mu2 = 2 * a_mu * dsquared

# step2
dx1 = dx_mu1 + dx_mu2
dmu = -1 * np.sum( dxmu1 + dxmu2, axis = 0 )

# step1
dx2 = 1 / N * np.ones(( N, D )) * dmu

# step0
dx = dx1 + dx2

In [ ]:
# unfold the variables stored in cache
xhat,gamma,xmu,ivar,sqrtvar,var,eps = cache

# step7
divar = np.sum(dxhat*xmu, axis=0)
dxmu1 = dxhat * ivar

# step6
dsqrtvar = -1 / ( sqrtvar ** 2 ) * divar

# step5
dvar = 0.5 * 1 / np.sqrt( var + eps ) * dsqrtvar

# step4
dsq = 1 / N * np.ones(( N, D )) * dvar

# step3
dxmu2 = 2 * xmu * dsq

# step2
dx1 = dxmu1 + dxmu2
dmu = -1 * np.sum( dxmu1 + dxmu2, axis = 0 )

# step1
dx2 = 1. /N * np.ones(( N, D )) * dmu

# step0
dx = dx1 + dx2

# return dx, dgamma, dbeta

In [49]:
# Check the training-time forward pass by checking means and variances
# of features both before and after batch normalization

# Simulate the forward pass for a two-layer network
N, D1, D2, D3 = 200, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print( 'Before batch normalization:' )
print( '  means: ', a.mean(axis = 0) )
print( '  stds: ', a.std(axis = 0) )

# Means should be close to zero and stds close to one
print( 'After batch normalization (gamma=1, beta=0)' )
a_norm, _ = layers.batchnorm_forward(a, np.ones(D3), np.zeros(D3), {'mode': 'train'})
print( '  mean: ', a_norm.mean(axis = 0) )
print( '  std: ', a_norm.std(axis = 0) )

# Now means should be close to beta and stds close to gamma
gamma = np.asarray([1.0, 2.0, 3.0])
beta = np.asarray([11.0, 12.0, 13.0])
a_norm, _ = layers.batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print( 'After batch normalization (nontrivial gamma, beta)' )
print( '  means: ', a_norm.mean(axis = 0) )
print( '  stds: ', a_norm.std(axis = 0) )

Before batch normalization:
  means:  [ -0.64667201  30.04606593   2.83172773]
  stds:  [ 42.63852309  27.89901264  36.58375319]
After batch normalization (gamma=1, beta=0)
  mean:  [ -2.49800181e-17  -1.34545153e-16  -7.19216353e-17]
  std:  [ 1.          0.99999999  1.        ]
After batch normalization (nontrivial gamma, beta)
  means:  [ 11.  12.  13.]
  stds:  [ 1.          1.99999999  2.99999999]


In [50]:
# Check the test-time forward pass by running the training-time
# forward pass many times to warm up the running averages, and then
# checking the means and variances of activations after a test-time
# forward pass.

N, D1, D2, D3 = 200, 50, 60, 3
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)

bn_param = {'mode': 'train'}
gamma = np.ones(D3)
beta = np.zeros(D3)
for t in range(50):
    X = np.random.randn(N, D1)
    a = np.maximum(0, X.dot(W1)).dot(W2)
    layers.batchnorm_forward(a, gamma, beta, bn_param)

bn_param['mode'] = 'test'
X = np.random.randn(N, D1)
a = np.maximum(0, X.dot(W1)).dot(W2)
a_norm, _ = layers.batchnorm_forward(a, gamma, beta, bn_param)

# Means should be close to zero and stds close to one, but will be
# noisier than training-time forward passes.
print( 'After batch normalization (test-time):' )
print( '  means: ', a_norm.mean(axis=0) )
print( '  stds: ', a_norm.std(axis=0) )

After batch normalization (test-time):
  means:  [-0.59374536 -1.71721708  1.17451839]
  stds:  [ 1.00717455  1.10634525  1.06992484]


## Batch Normalization: backward

Now implement the backward pass for batch normalization in the function `batchnorm_backward`.

To derive the backward pass you should write out the computation graph for batch normalization and backprop through each of the intermediate nodes. Some intermediates may have multiple outgoing branches; make sure to sum gradients across these branches in the backward pass. Once you have finished, run the following to numerically check your backward pass.

In [53]:
# Gradient check batchnorm backward pass

N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
fx = lambda x: layers.batchnorm_forward(x, gamma, beta, bn_param)[0]
fg = lambda a: layers.batchnorm_forward(x, gamma, beta, bn_param)[0]
fb = lambda b: layers.batchnorm_forward(x, gamma, beta, bn_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma, dout)
db_num = eval_numerical_gradient_array(fb, beta, dout)

_, cache = layers.batchnorm_forward(x, gamma, beta, bn_param)
dx, dgamma, dbeta = layers.batchnorm_backward(dout, cache)
print( 'dx error: ', rel_error(dx_num, dx) )
print( 'dgamma error: ', rel_error(da_num, dgamma) )
print( 'dbeta error: ', rel_error(db_num, dbeta) )

dx error:  8.30494765027e-09
dgamma error:  7.50622733363e-11
dbeta error:  2.18764424699e-11


## Batch Normalization: alternative backward
In class we talked about two different implementations for the sigmoid backward pass. One strategy is to write out a computation graph composed of simple operations and backprop through all intermediate values. Another strategy is to work out the derivatives on paper. For the sigmoid function, it turns out that you can derive a very simple formula for the backward pass by simplifying gradients on paper.

Surprisingly, it turns out that you can also derive a simple expression for the batch normalization backward pass if you work out derivatives on paper and simplify. After doing so, implement the simplified batch normalization backward pass in the function `batchnorm_backward_alt` and compare the two implementations by running the following. Your two implementations should compute nearly identical results, but the alternative implementation should be a bit faster.

NOTE: You can still complete the rest of the assignment if you don't figure this part out, so don't worry too much if you can't get it.

In [ ]:
N, D = 100, 500
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
out, cache = batchnorm_forward(x, gamma, beta, bn_param)

t1 = time.time()
dx1, dgamma1, dbeta1 = batchnorm_backward(dout, cache)
t2 = time.time()
dx2, dgamma2, dbeta2 = batchnorm_backward_alt(dout, cache)
t3 = time.time()

print( 'dx difference: ', rel_error(dx1, dx2) )
print( 'dgamma difference: ', rel_error(dgamma1, dgamma2) )
print( 'dbeta difference: ', rel_error(dbeta1, dbeta2) )
print( 'speedup: %.2fx' % ((t2 - t1) / (t3 - t2)) )

## Fully Connected Nets with Batch Normalization
Now that you have a working implementation for batch normalization, go back to your `FullyConnectedNet` in the file `cs2312n/classifiers/fc_net.py`. Modify your implementation to add batch normalization.

Concretely, when the flag `use_batchnorm` is `True` in the constructor, you should insert a batch normalization layer before each ReLU nonlinearity. The outputs from the last layer of the network should not be normalized. Once you are done, run the following to gradient-check your implementation.

HINT: You might find it useful to define an additional helper layer similar to those in the file `cs231n/layer_utils.py`. If you decide to do so, do it in the file `cs231n/classifiers/fc_net.py`.

In [ ]:
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D)
y = np.random.randint(C, size=(N,))

for reg in [0, 3.14]:
    print( 'Running check with reg = ', reg )
    model = FullyConnectedNet([H1, H2], input_dim=D, num_classes=C,
                            reg=reg, weight_scale=5e-2, dtype=np.float64,
                            use_batchnorm=True)

    loss, grads = model.loss(X, y)
    print( 'Initial loss: ', loss )

    for name in sorted(grads):
        f = lambda _: model.loss(X, y)[0]
        grad_num = eval_numerical_gradient(f, model.params[name], verbose=False, h=1e-5)
        print( '%s relative error: %.2e' % (name, rel_error(grad_num, grads[name])) )
    
    if reg == 0: print()

# Batchnorm for deep networks
Run the following to train a six-layer network on a subset of 1000 training examples both with and without batch normalization.

In [ ]:
# Try training a very deep net with batchnorm
hidden_dims = [100, 100, 100, 100, 100]

num_train = 1000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

weight_scale = 2e-2
bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, use_batchnorm=True)
model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, use_batchnorm=False)

bn_solver = Solver(bn_model, small_data,
                num_epochs=10, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-3,
                },
                verbose=True, print_every=200)
bn_solver.train()

solver = Solver(model, small_data,
                num_epochs=10, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-3,
                },
                verbose=True, print_every=200)
solver.train()

Run the following to visualize the results from two networks trained above. You should find that using batch normalization helps the network to converge much faster.

In [ ]:
plt.subplot(3, 1, 1)
plt.title('Training loss')
plt.xlabel('Iteration')

plt.subplot(3, 1, 2)
plt.title('Training accuracy')
plt.xlabel('Epoch')

plt.subplot(3, 1, 3)
plt.title('Validation accuracy')
plt.xlabel('Epoch')

plt.subplot(3, 1, 1)
plt.plot(solver.loss_history, 'o', label='baseline')
plt.plot(bn_solver.loss_history, 'o', label='batchnorm')

plt.subplot(3, 1, 2)
plt.plot(solver.train_acc_history, '-o', label='baseline')
plt.plot(bn_solver.train_acc_history, '-o', label='batchnorm')

plt.subplot(3, 1, 3)
plt.plot(solver.val_acc_history, '-o', label='baseline')
plt.plot(bn_solver.val_acc_history, '-o', label='batchnorm')
  
for i in [1, 2, 3]:
    plt.subplot(3, 1, i)
    plt.legend(loc='upper center', ncol=4)

    plt.gcf().set_size_inches(15, 15)
plt.show()

# Batch normalization and initialization
We will now run a small experiment to study the interaction of batch normalization and weight initialization.

The first cell will train 8-layer networks both with and without batch normalization using different scales for weight initialization. The second layer will plot training accuracy, validation set accuracy, and training loss as a function of the weight initialization scale.

In [ ]:
# Try training a very deep net with batchnorm
hidden_dims = [50, 50, 50, 50, 50, 50, 50]

num_train = 1000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

bn_solvers = {}
solvers = {}
weight_scales = np.logspace(-4, 0, num=20)
for i, weight_scale in enumerate(weight_scales):
    print( 'Running weight scale %d / %d' % (i + 1, len(weight_scales)) )
    bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, use_batchnorm=True)
    model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, use_batchnorm=False)

    bn_solver = Solver(bn_model, small_data,
                  num_epochs=10, batch_size=50,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 1e-3,
                  },
                  verbose=False, print_every=200)
    bn_solver.train()
    bn_solvers[weight_scale] = bn_solver

    solver = Solver(model, small_data,
                  num_epochs=10, batch_size=50,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 1e-3,
                  },
                  verbose=False, print_every=200)
    solver.train()
    solvers[weight_scale] = solver

In [ ]:
# Plot results of weight scale experiment
best_train_accs, bn_best_train_accs = [], []
best_val_accs, bn_best_val_accs = [], []
final_train_loss, bn_final_train_loss = [], []

for ws in weight_scales:
    best_train_accs.append(max(solvers[ws].train_acc_history))
    bn_best_train_accs.append(max(bn_solvers[ws].train_acc_history))
  
    best_val_accs.append(max(solvers[ws].val_acc_history))
    bn_best_val_accs.append(max(bn_solvers[ws].val_acc_history))
  
    final_train_loss.append(np.mean(solvers[ws].loss_history[-100:]))
    bn_final_train_loss.append(np.mean(bn_solvers[ws].loss_history[-100:]))

plt.subplot(3, 1, 1)
plt.title('Best val accuracy vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Best val accuracy')
plt.semilogx(weight_scales, best_val_accs, '-o', label='baseline')
plt.semilogx(weight_scales, bn_best_val_accs, '-o', label='batchnorm')
plt.legend(ncol=2, loc='lower right')

plt.subplot(3, 1, 2)
plt.title('Best train accuracy vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Best training accuracy')
plt.semilogx(weight_scales, best_train_accs, '-o', label='baseline')
plt.semilogx(weight_scales, bn_best_train_accs, '-o', label='batchnorm')
plt.legend()

plt.subplot(3, 1, 3)
plt.title('Final training loss vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Final training loss')
plt.semilogx(weight_scales, final_train_loss, '-o', label='baseline')
plt.semilogx(weight_scales, bn_final_train_loss, '-o', label='batchnorm')
plt.legend()

plt.gcf().set_size_inches(10, 15)
plt.show()

# Question:
Describe the results of this experiment, and try to give a reason why the experiment gave the results that it did.

# Answer:


## Reference

Standford CS231n: Convolutional Neural Networks for Visual Recognition

- [Course Notes: Backpropagation](http://cs231n.github.io/optimization-2/)
- [Course Notes: Setting Up the Architecture](http://cs231n.github.io/neural-networks-1/)
- [Course Notes: Setting Up the Data and Loss](http://cs231n.github.io/neural-networks-2/)
- [Course Notes: Learning and Evaluation](http://cs231n.github.io/neural-networks-3/)
- [Course Notes: Putting it together: Minimal Neural Network Case Study](http://cs231n.github.io/neural-networks-case-study/)
- [Course Home Page](http://cs231n.stanford.edu/index.html)
- [Course Github](https://github.com/cs231n/cs231n.github.io)
- [Course Youtube](https://www.youtube.com/watch?v=NfnWJUyUJYU&list=PLkt2uSq6rBVctENoVBg1TpCC7OQi31AlC)
- [Blog: Batch Normalization Layer](https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html).